In [249]:
from playwright.async_api import async_playwright
from playwright_stealth import stealth_sync , stealth_async


In [250]:

pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)
# browser = await pw.chromium.launch()


In [251]:
import os
os.makedirs('playwright/.auth', exist_ok=True)
if os.path.exists('playwright/.auth/state.json'):
    context = await browser.new_context(storage_state="playwright/.auth/state.json")
    page = await context.new_page()


In [252]:
async def extract_info(page):
    container = await page.query_selector('.e1usz6i03')
    element = await container.query_selector('#phone-number-content')
    phone_number  = await element.inner_text()
    return phone_number

In [253]:
async def search_person(page, firstName, lastName, middleName="", city=None, state=None):
    # https://www.spokeo.com/Samama-Mahmud?middle_name=B
    url = f"https://www.spokeo.com/{firstName}-{lastName}?middle_name={middleName}"
    await page.goto(url)    
    link_elements = await page.query_selector_all('.e1ndw42t0')
    # links = [
    #     link_element.get_attribute('href')
    #     for link_element in link_elements
    # ]
    links = []
    for link_element in link_elements:
        link = await link_element.get_attribute('href')
        links.append(link)

    
    return links

In [254]:
firstName = "Samama"
lastName = "Mahmud"
middleName = "B"
user_page  = await search_person(page,firstName, lastName,middleName)
print(user_page)

['/Samama-Mahmud/Wisconsin/Milwaukee/p2015111502099087352946730813837']


In [255]:
url_user = "https://www.spokeo.com/Samama-Mahmud/Wisconsin/Milwaukee/p2015111502099087352946730813837"
await page.goto(url_user)  



<Response url='https://www.spokeo.com/Samama-Mahmud/Wisconsin/Milwaukee/p2015111502099087352946730813837' request=<Request url='https://www.spokeo.com/Samama-Mahmud/Wisconsin/Milwaukee/p2015111502099087352946730813837' method='GET'>>

In [256]:
# phone_number = await extract_info(page)
# phone_number

container = await page.query_selector('.e1usz6i03')
container

<JSHandle preview=JSHandle@node>

In [257]:
el = await container.query_selector('#phone-number-content')

In [258]:
await el.inner_text()

'(847) 581-0171'

In [259]:
async def preson_search(page, firstName, lastName, middleName="", city=None, state=None):
    # https://www.spokeo.com/Samama-Mahmud?middle_name=B
    url = f"https://www.spokeo.com/{firstName}-{lastName}?middle_name={middleName}"
    await page.goto(url)    
    link_elements = await page.query_selector_all('.e1ndw42t0')
    # links = [
    #     link_element.get_attribute('href')
    #     for link_element in link_elements
    # ]
    links = []
    for link_element in link_elements:
        link = await link_element.get_attribute('href')
        links.append(f"https://www.spokeo.com{link}")
        
    phones = []
    for link in links:
        await page.goto(link)
        try:
            phone_number = await extract_info(page)
            phones.append(phone_number)
        except Exception as e:
            print(e,"==>", link)
            pass
 
    return phones

In [260]:
firstName = "Samama"
lastName = "Mahmud"
middleName = "B"
user_page  = await preson_search(page,firstName, lastName,middleName)
user_page

['(847) 581-0171']

# Leads Analysis


In [261]:
#load data into a dataframe
import pandas as pd
df = pd.read_csv('leads_not_found.csv')
df.shape

(107, 34)

In [262]:
df.head()

,id,case_id,first_name,last_name,middle_name,court_code,age,year_of_birth,email,phone,...,notes,violation,court,accidentCheckbox,commercialDriverLicence,ticket_img,user_id,report,details,cloudtalk_upload
0,NaN,DICKERSON__JOHN_E_63_01_02_2024,JOHN,DICKERSON,E,temp,63,1961,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
1,NaN,BIGGS__EUGENE_L_25_01_02_2024,EUGENE,BIGGS,L,temp,25,1999,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
2,NaN,24TR60000100,NATASHA,TAYLOR,C,IL_COOK,47,1977,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
3,NaN,24TR60000091,LORENZO,GERARDO,A,IL_COOK,35,1989,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
4,NaN,24TR50000090,JENNIFER,HENKE,L,IL_COOK,42,1982,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False


In [263]:
df['middle_name'] = df['middle_name'].fillna('')
df['first_name'] = df['first_name'].fillna('')
df['last_name'] = df['last_name'].fillna('')
df

,id,case_id,first_name,last_name,middle_name,court_code,age,year_of_birth,email,phone,...,notes,violation,court,accidentCheckbox,commercialDriverLicence,ticket_img,user_id,report,details,cloudtalk_upload
0,NaN,DICKERSON__JOHN_E_63_01_02_2024,JOHN,DICKERSON,E,temp,63,1961,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
1,NaN,BIGGS__EUGENE_L_25_01_02_2024,EUGENE,BIGGS,L,temp,25,1999,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
2,NaN,24TR60000100,NATASHA,TAYLOR,C,IL_COOK,47,1977,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
3,NaN,24TR60000091,LORENZO,GERARDO,A,IL_COOK,35,1989,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
4,NaN,24TR50000090,JENNIFER,HENKE,L,IL_COOK,42,1982,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,NaN,230257248,NATHAN,GLODECK,STANLEY,CT21,18,2005,NaN,NaN,...,NaN,NaN,NaN,False,False,230257248.png,NaN,NaN,NaN,False
103,NaN,220460163,AMORION,OLIPHANT,JAVON,CT21,19,2004,NaN,NaN,...,NaN,NaN,NaN,False,False,220460163.png,NaN,NaN,NaN,False
104,NaN,180458951,BARBARA,WILSON,V,CT21,43,1980,NaN,NaN,...,NaN,NaN,NaN,False,False,180458951.png,NaN,NaN,NaN,False
105,NaN,GREER__LAMAR_L_28_01_01_2024,LAMAR,GREER,L,temp,28,1996,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False


In [264]:
# df['phone_numbers_spokeo'] = df.apply(lambda x: preson_search(x['first_name'], x['last_name'], x['middle_name']), axis=1)
phone_numbers_spokeo = []
# df = df.head(2)

In [265]:
for index, row in df.iterrows():
    try:
        result = await preson_search(page, row['first_name'], row['last_name'], row['middle_name'])
        print(f"user {row['first_name']} ==> ", result)
    except Exception as e:
        print(e)
        result = []

    phone_numbers_spokeo.append(result)

'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/California/Garden-Grove/p2018020712336929942701038168173
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/Texas/Austin/p2016110406511622702879694864356
user JOHN ==>  ['(714) 894-6399', '(803) 356-1433 +4 phones', '(480) 839-4121 +3 phones', '(803) 371-9060 +4 phones', '(904) 744-7825 +4 phones', '(803) 822-8832 +6 phones', '(302) 258-8358 +1 phone', '(318) 934-3491 +3 phones', '(435) 729-9560 +5 phones', '(803) 920-0165 +8 phones', '(423) 383-8624 +7 phones', '(513) 594-7331 +5 phones', '(770) 808-4510', '(973) 627-6102', '(302) 258-8999 +2 phones', '(714) 850-0688', '(973) 839-2535', '(770) 386-9157', '(402) 758-1233 +1 phone', '(317) 796-4992 +1 phone', '(440) 821-3073 +4 phones', '(712) 534-3549', '(540) 478-3578 +7 phones', '(315) 662-8046 +3 phones', '(423) 383-8624 +2 phones', '(919) 868-1573 +1 phone', '(318) 949-2063 +1 phone', 'See availab

In [266]:
phone_numbers_spokeo

[['(714) 894-6399',
  '(803) 356-1433 +4 phones',
  '(480) 839-4121 +3 phones',
  '(803) 371-9060 +4 phones',
  '(904) 744-7825 +4 phones',
  '(803) 822-8832 +6 phones',
  '(302) 258-8358 +1 phone',
  '(318) 934-3491 +3 phones',
  '(435) 729-9560 +5 phones',
  '(803) 920-0165 +8 phones',
  '(423) 383-8624 +7 phones',
  '(513) 594-7331 +5 phones',
  '(770) 808-4510',
  '(973) 627-6102',
  '(302) 258-8999 +2 phones',
  '(714) 850-0688',
  '(973) 839-2535',
  '(770) 386-9157',
  '(402) 758-1233 +1 phone',
  '(317) 796-4992 +1 phone',
  '(440) 821-3073 +4 phones',
  '(712) 534-3549',
  '(540) 478-3578 +7 phones',
  '(315) 662-8046 +3 phones',
  '(423) 383-8624 +2 phones',
  '(919) 868-1573 +1 phone',
  '(318) 949-2063 +1 phone',
  'See available information'],
 ['(910) 461-0094 +5 phones',
  '(402) 812-1311 +7 phones',
  '(618) 724-7335',
  '(812) 882-9223',
  'See available information',
  '(303) 884-3599 +3 phones',
  '(540) 797-5004 +7 phones',
  '(901) 233-1586 +4 phones'],
 ['(714) 67

In [267]:
df['phone_numbers_spokeo'] = phone_numbers_spokeo


In [276]:
df['phone_numbers_spokeo'][4]


[]

In [269]:
df.to_csv('phone_numbers_spokeo.csv', index=False)

In [270]:
await browser.close()
await pw.stop()